安裝各項依賴

In [ ]:
!pip install openai-whisper
!apt install ffmpeg
!pip install deepl
!pip install yt-dlp
!pip install openai==0.28

主要程式





禁止使用deepL + OpenAI翻譯

不好用，沒有比較好

In [ ]:

import openai
from google.colab import userdata

openai.api_key = userdata.get("openai")

import os
import subprocess
import time
import whisper

model = whisper.load_model("medium")

def translate_text(text, target_language="繁體中文"):

    translation_prompt = f"請將以下語句翻譯成流暢的{target_language}:\n{text}"

    improvement_prompt = "之後你必須根據上一句翻譯的語句來推測你的這一句如何變得更通順流暢"

    prompt = translation_prompt + "\n" + improvement_prompt

    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=500
    )

    return response.choices[0].text

# 使用 yt-dlp 獲取 YouTube 媒體流地址
def get_stream_url(youtube_url):
    command = ["yt-dlp", "-g", youtube_url]
    process = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    if process.returncode != 0:
        print(f"錯誤: 無法獲取流地址。詳情：{process.stderr}")
        return None
    return process.stdout.strip()

# 錄製音頻的函數
def record_audio(stream_url, duration, output_filename):
    try:
        command = [
            "ffmpeg", "-i", stream_url, "-t", str(duration),
            "-ac", "1", "-ar", "16000", "-y", output_filename
        ]
        subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, check=True)
    except subprocess.CalledProcessError as e:
        print(f"錯誤: 無法錄製音頻。詳情：{e.stderr.decode()}")
        return False
    return True

# 轉寫音頻的函數
def transcribe_audio(file_path, language="japanese"):
    try:
        result = model.transcribe(file_path, language=language)
        return result["text"]
    except RuntimeError as e:
        print(f"錯誤: 無法轉寫音頻。詳情：{e}")
        return None

# 主流程
def main(youtube_url, segment_duration, total_duration):
    stream_url = get_stream_url(youtube_url)
    if not stream_url:
        print("無法獲取 YouTube 直播流地址。")
        return

    for segment in range(0, total_duration, segment_duration):
        filename = f"output{segment}.wav"
        if record_audio(stream_url, segment_duration, filename):
            text = transcribe_audio(filename)
            if text:
                translated_text = translate_text(text)
                print(f"Segment {segment}: {translated_text}")
            else:
                print(f"無法轉寫段落 {segment}。")
            os.remove(filename)  # 清理：刪除音頻文件
        else:
            print(f"錄製段落 {segment} 失敗。")
        time.sleep(segment_duration)

# 配置參數
YOUTUBE_URL = "https://www.youtube.com/watch?v=GGeSozh7Yzw" # YouTube 直播頁面 URL
SEGMENT_DURATION = 10  # 每個音頻片段的長度，單位：秒
TOTAL_DURATION = 6000   # 總錄製時間，單位：秒

if __name__ == "__main__":
    main(YOUTUBE_URL, SEGMENT_DURATION, TOTAL_DURATION)